In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import googlemaps

Google Places API used just for this project and removed once completed for best security practices. Moreover, API use was filtered to be used just from 1 IP

In [227]:
# hidden_cell
gmaps = googlemaps.Client(key='AIzxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxPTo')

Setting the web url. It'll be hidden on the code in order to avoid its use due to legal terms on the website. The legitimacy of using Web scraping to collect data still unclear.

In [ ]:
# @hidden_cell
url_base = 'https://www.T-A.ie'

### <a id="data_ac">Data Acquisition</a>

Firstly we need to collect the list of all restaurants in Dublin city.

In [140]:
TA_restaurants = pd.DataFrame(columns=['Name', 'Ranking', 'Link'])
pages = list(range(0, 2310, 30))
i=0
for page in pages:
    print(page)
    url = '{}/Restaurants-g186605-oa{}-Dublin_County_Dublin.html#EATERY_LIST_CONTENTS'.format (url_base, str(page))
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    source = requests.get(url, headers=headers).text
    soup = BeautifulSoup(source, 'lxml')
    amusements_soup = BeautifulSoup(source,"html.parser")
    time.sleep(1)
    TA_rest = pd.DataFrame(columns=['Name', 'Ranking', 'Link'])

    for item in amusements_soup.findAll('div',{'class':'wQjYiB7z'}):
        sub_text = item.text
        if sub_text[0].isdigit():
            sub_text2 = sub_text.strip().split('.', 1)
            TA_rest.loc[i, 'Ranking'] = sub_text2[0].strip()
            TA_rest.loc[i, 'Name'] = sub_text2[1:][0].strip()
            sub_items = item.findAll('a', href=True)
            for sub_item in sub_items:
                if sub_item.text:
                    TA_rest.loc[i, 'Link'] = sub_item['href']
                    i+=1
                else:
                    i+=1
                    continue
        else:
            continue    
            
    TA_restaurants = TA_restaurants.append(TA_rest)
    time.sleep(1)

0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
2010
2040
2070
2100
2130
2160
2190
2220
2250
2280


In [129]:
#TA_restaurants.reset_index(drop=True, inplace=True)

In [141]:
TA_restaurants.describe()

,Name,Ranking,Link
count,2297,2297,2297
unique,2118,2297,2297
top,Subway,1346,/Restaurant_Review-g186605-d8128966-Reviews-Ku...
freq,40,1,1


In [81]:
TA_restaurants.head()

,Name,Ranking,Link
0,Tang Cafe,1,/Restaurant_Review-g186605-d10387074-Reviews-T...
1,Taza,2,/Restaurant_Review-g186605-d17181958-Reviews-T...
2,Glovers Alley,3,/Restaurant_Review-g186605-d13477650-Reviews-G...
3,Darkey Kelly's Bar & Restaurant,4,/Restaurant_Review-g186605-d6403998-Reviews-Da...
4,Mulberry Garden,5,/Restaurant_Review-g186605-d2239110-Reviews-Mu...


In [145]:
#TA_restaurants.to_csv('TA_restaurants_ranking3.csv', index=False)

### TA_restaurants = pd.read_csv('TA_restaurants_ranking3.csv')

In [350]:
TA_restaurants.Name.is_unique

False

In [7]:
TA_Links = TA_restaurants.copy()

In [352]:
TA_Links.describe()

,Name,Ranking,Link
count,2297,2297,2297
unique,2118,2297,2297
top,Subway,1346,/Restaurant_Review-g186605-d8128966-Reviews-Ku...
freq,40,1,1


In [8]:
TA_Links.drop_duplicates(subset=['Name'], keep="first", inplace=True)

In [9]:
TA_Links.describe()

,Ranking
count,2118.000000
mean,1094.275732
std,647.507116
min,1.000000
25%,533.250000
50%,1082.500000
75%,1637.750000
max,2295.000000


In [10]:
TA_Links.reset_index(drop=True, inplace=True)

In [80]:
TA_Links.head()

,Name,Ranking,Link
0,Tang Cafe,1,/Restaurant_Review-g186605-d10387074-Reviews-T...
1,Taza,2,/Restaurant_Review-g186605-d17181958-Reviews-T...
2,Glovers Alley,3,/Restaurant_Review-g186605-d13477650-Reviews-G...
3,Darkey Kelly's Bar & Restaurant,4,/Restaurant_Review-g186605-d6403998-Reviews-Da...
4,Mulberry Garden,5,/Restaurant_Review-g186605-d2239110-Reviews-Mu...


#### Getting the details of each restaurant using the column 'Link'

In [500]:
TA_details = pd.DataFrame(columns = ['Reviews', 'Rating', 'Price', 'Cuisines', 'Address', 'Phone', 'Link'])
TA_details_errors = []

In [501]:
i = 0
prices = ['€€€€', '$$ - $$$', '€']
for i in range (0, 2101):
    space = '     '
    print('{} {}'.format(i, space), end="\r", flush=True)

    link = TA_Links.loc[i, 'Link']
    url = url_base + link

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    source = requests.get(url, headers=headers).text
    soup = BeautifulSoup(source, 'lxml')
    amusements_soup = BeautifulSoup(source,"html.parser")

    sub_list = []
    TA_detail = pd.DataFrame(columns = ['Reviews', 'Rating', 'Price', 'Cuisines', 'Address', 'Phone', 'Link'])
    for item in amusements_soup.findAll('div',{'class':'restaurants-detail-top-info-TopInfo__infoRow--2m8Mx'}):
        sub_items = item.findAll('a')
        for sub_item in sub_items:
            sub_list.append(sub_item.text)
    try:
        TA_detail.loc[i, 'Reviews'] = sub_list[0].split()[0].strip()
        if sub_list[2] == '€€€€':
            TA_detail.loc[i, 'Price'] = 'Expensive'
        elif sub_list[2] == '$$ - $$$':
            TA_detail.loc[i, 'Price'] = 'Average'
        elif sub_list[2] == '€':
            TA_detail.loc[i, 'Price'] = 'Cheap'
        else:
            TA_detail.loc[i, 'Price'] = 'N/A'
        
        if sub_list[-1] == '+ Add hours':      
            if sub_list[-2] == '+ Add phone number':
                if sub_list[2] not in prices:
                    TA_detail.loc[i, 'Price'] = 'N/A'
                    TA_detail.loc[i, 'Cuisines'] = sub_list[2:-3]
                    TA_detail.loc[i, 'Address'] = sub_list[-3]
                    TA_detail.loc[i, 'Phone'] = 'N/A'
                else:
                    TA_detail.loc[i, 'Cuisines'] = sub_list[3:-3]
                    TA_detail.loc[i, 'Address'] = sub_list[-3]
                    TA_detail.loc[i, 'Phone'] = 'N/A'
            elif sub_list[-2] == '+ Add your menu':  
                if sub_list[-3] == '+ Add website':
                    if sub_list[2] not in prices:
                        TA_detail.loc[i, 'Price'] = 'N/A'
                        TA_detail.loc[i, 'Cuisines'] = sub_list[2:-5]
                        TA_detail.loc[i, 'Address'] = sub_list[-5]
                        TA_detail.loc[i, 'Phone'] = sub_list[-4]
                    else:
                        TA_detail.loc[i, 'Cuisines'] = sub_list[3:-5]
                        TA_detail.loc[i, 'Address'] = sub_list[-5]
                        TA_detail.loc[i, 'Phone'] = sub_list[-4]                               
                else:                    
                    if sub_list[2] not in prices:
                        TA_detail.loc[i, 'Price'] = 'N/A'
                        TA_detail.loc[i, 'Cuisines'] = sub_list[2:-4]
                        TA_detail.loc[i, 'Address'] = sub_list[-4]
                        TA_detail.loc[i, 'Phone'] = sub_list[-3]
                    else:
                        TA_detail.loc[i, 'Cuisines'] = sub_list[3:-4]
                        TA_detail.loc[i, 'Address'] = sub_list[-4]
                        TA_detail.loc[i, 'Phone'] = sub_list[-3]
            elif sub_list[-2] != '+ Add phone number' and sub_list[-2] != '+ Add your menu':
                if sub_list[2] not in prices:
                    TA_detail.loc[i, 'Price'] = 'N/A'
                    TA_detail.loc[i, 'Cuisines'] = sub_list[2:-3]
                    TA_detail.loc[i, 'Address'] = sub_list[-3]
                    TA_detail.loc[i, 'Phone'] = sub_list[-2]
                else:
                    TA_detail.loc[i, 'Cuisines'] = sub_list[3:-3]
                    TA_detail.loc[i, 'Address'] = sub_list[-3]
                    TA_detail.loc[i, 'Phone'] = sub_list[-2]                

                
        elif sub_list[-1] == '+ Add your menu': 
            if sub_list[-2] == '+ Add phone number':
                if sub_list[2] not in prices:
                    TA_detail.loc[i, 'Price'] = 'N/A'
                    TA_detail.loc[i, 'Cuisines'] = sub_list[2:-3]
                    TA_detail.loc[i, 'Address'] = sub_list[-3]
                    TA_detail.loc[i, 'Phone'] = 'N/A'
                else:
                    TA_detail.loc[i, 'Cuisines'] = sub_list[3:-3]
                    TA_detail.loc[i, 'Address'] = sub_list[-3]
                    TA_detail.loc[i, 'Phone'] = 'N/A'
            elif sub_list[-2] == '+ Add website':
                if sub_list[-3] == '+ Add phone number':
                    if sub_list[2] not in prices:
                        TA_detail.loc[i, 'Price'] = 'N/A'
                        TA_detail.loc[i, 'Cuisines'] = sub_list[2:-4]
                        TA_detail.loc[i, 'Address'] = sub_list[-4]
                        TA_detail.loc[i, 'Phone'] = 'N/A'
                    else:
                        TA_detail.loc[i, 'Cuisines'] = sub_list[3:-4]
                        TA_detail.loc[i, 'Address'] = sub_list[-4]
                        TA_detail.loc[i, 'Phone'] = 'N/A'      
                else:      
                    if sub_list[2] not in prices:
                        TA_detail.loc[i, 'Price'] = 'N/A'
                        TA_detail.loc[i, 'Cuisines'] = sub_list[2:-4]
                        TA_detail.loc[i, 'Address'] = sub_list[-4]
                        TA_detail.loc[i, 'Phone'] = sub_list[-3]
                    else:
                        TA_detail.loc[i, 'Cuisines'] = sub_list[3:-4]
                        TA_detail.loc[i, 'Address'] = sub_list[-4]
                        TA_detail.loc[i, 'Phone'] = sub_list[-3] 
            else:
                if sub_list[2] not in prices:
                    TA_detail.loc[i, 'Price'] = 'N/A'
                    TA_detail.loc[i, 'Cuisines'] = sub_list[2:-3]
                    TA_detail.loc[i, 'Address'] = sub_list[-3]
                    TA_detail.loc[i, 'Phone'] = sub_list[-2]
                else:
                    TA_detail.loc[i, 'Cuisines'] = sub_list[3:-3]
                    TA_detail.loc[i, 'Address'] = sub_list[-3]
                    TA_detail.loc[i, 'Phone'] = sub_list[-2]
                                            
        else:
            if sub_list[2] not in prices:
                TA_detail.loc[i, 'Price'] = 'N/A'
                TA_detail.loc[i, 'Cuisines'] = sub_list[2:-2]
                TA_detail.loc[i, 'Address'] = sub_list[-2]
                TA_detail.loc[i, 'Phone'] = sub_list[-1]
            else:
                TA_detail.loc[i, 'Cuisines'] = sub_list[3:-2]
                TA_detail.loc[i, 'Address'] = sub_list[-2]
                TA_detail.loc[i, 'Phone'] = sub_list[-1] 
            
        sub_list2 = []
        for item in amusements_soup.findAll('div',{'class':'restaurants-detail-overview-cards-RatingsOverviewCard__primaryRatingRow--VhEsu restaurants-detail-overview-cards-RatingsOverviewCard__cx_brand_refresh_phase2--1d7i-'}):
            sub_list2.append(item.text)
        TA_detail.loc[i, 'Rating'] = sub_list2[0].split()[0]
        TA_detail.loc[i, 'Link'] = link

        TA_details = TA_details.append(TA_detail)
        #if i%10 == 0:
        #    time.sleep(1)
        #else:
        #    continue
    except:
        TA_details_errors.append(link)
        continue
        
print('Completed')

Completed 


In [ ]:
# TA_details.to_csv('TA_details_1.csv', index=False)

In [5]:
TA_details = pd.read_csv('TA_details_updated.csv')

In [577]:
print ('There were {} restaurant with no data to be added'. format (len(TA_details_errors)))

There were 163 restaurant with no data to be added


Checking the Ranking of the restaurants which have given error

In [611]:
List_ranking_errors = []
for link in TA_details_errors:
    Ranking = TA_Links[TA_Links.Link == link].Ranking.values
    List_ranking_errors.append(list(Ranking)[0])
List_ranking_errors.sort()

In [616]:
print('{},....'.format (List_ranking_errors[0:11]))

['2046', '2048', '2050', '2051', '2052', '2053', '2054', '2056', '2057', '2058', '2059'],....


These restaurant with a Ranking above 2000 are not relevant within the project scope

#### Merging Dataframes ( List of restaurants collected using the first script and the details of each one)

Checking first that there are not duplicates

In [14]:
TA_details.Link.is_unique

True

In [13]:
TA_Links.Link.is_unique

True

In [15]:
TA_complete = pd.merge(TA_Links, TA_details, on='Link' )

In [16]:
TA_complete.reset_index(drop=True, inplace=True)

In [17]:
TA_complete.dtypes

Name         object
Ranking       int64
Link         object
Reviews       int64
Rating      float64
Price        object
Cuisines     object
Address      object
Phone        object
dtype: object

In [18]:
TA_complete.head()

,Name,Ranking,Link,Reviews,Rating,Price,Cuisines,Address,Phone
0,Tang Cafe,1,/Restaurant_Review-g186605-d10387074-Reviews-T...,502,5.0,Cheap,"['Cafe', 'European', 'Healthy']","23c Dawson Street, Dublin D02 PW18 Ireland",+353 86 391 5401
1,Taza,2,/Restaurant_Review-g186605-d17181958-Reviews-T...,178,5.0,Average,"['Indian', 'Asian', 'Pakistani']","2 Ardcollum Avenue Artane, Dublin D05 XW88 Ire...",+353 1 558 2866
2,Glovers Alley,3,/Restaurant_Review-g186605-d13477650-Reviews-G...,193,5.0,Expensive,"['Irish', 'European', 'Vegetarian Friendly']","128 Stephen’s Green Fitzwilliam Hotel, Dublin ...",+353 1 244 0733
3,Darkey Kelly's Bar & Restaurant,4,/Restaurant_Review-g186605-d6403998-Reviews-Da...,1645,4.5,Average,"['Irish', 'Bar', 'European']","Fishamble Street ChristChruch, Dublin Ireland",+353 83 346 4682
4,Mulberry Garden,5,/Restaurant_Review-g186605-d2239110-Reviews-Mu...,909,4.5,Expensive,"['Irish', 'European', 'Contemporary']","Mulberry Lane Donnybrook, Dublin 04 Ireland",+353 1 269 3300


In [19]:
TA_complete.columns

Index(['Name', 'Ranking', 'Link', 'Reviews', 'Rating', 'Price', 'Cuisines',
       'Address', 'Phone'],
      dtype='object')

Ordening columns

In [20]:
TA_complete = TA_complete.reindex(columns=['Name', 'Ranking', 'Reviews', 'Rating', 'Price', 'Cuisines',
       'Address', 'Phone', 'Link'])

In [21]:
TA_complete.head()

,Name,Ranking,Reviews,Rating,Price,Cuisines,Address,Phone,Link
0,Tang Cafe,1,502,5.0,Cheap,"['Cafe', 'European', 'Healthy']","23c Dawson Street, Dublin D02 PW18 Ireland",+353 86 391 5401,/Restaurant_Review-g186605-d10387074-Reviews-T...
1,Taza,2,178,5.0,Average,"['Indian', 'Asian', 'Pakistani']","2 Ardcollum Avenue Artane, Dublin D05 XW88 Ire...",+353 1 558 2866,/Restaurant_Review-g186605-d17181958-Reviews-T...
2,Glovers Alley,3,193,5.0,Expensive,"['Irish', 'European', 'Vegetarian Friendly']","128 Stephen’s Green Fitzwilliam Hotel, Dublin ...",+353 1 244 0733,/Restaurant_Review-g186605-d13477650-Reviews-G...
3,Darkey Kelly's Bar & Restaurant,4,1645,4.5,Average,"['Irish', 'Bar', 'European']","Fishamble Street ChristChruch, Dublin Ireland",+353 83 346 4682,/Restaurant_Review-g186605-d6403998-Reviews-Da...
4,Mulberry Garden,5,909,4.5,Expensive,"['Irish', 'European', 'Contemporary']","Mulberry Lane Donnybrook, Dublin 04 Ireland",+353 1 269 3300,/Restaurant_Review-g186605-d2239110-Reviews-Mu...


Adding coordinates to each restaurant using Geopy

In [30]:
TA_complete_updated = TA_complete.copy()

In [26]:
from geopy.geocoders import Nominatim
import re

In [33]:
def getaddress(name):
    coordenates = []
    geolocator = Nominatim(user_agent="ie_explorer")
    location = geolocator.geocode(name)
    latitude = location.latitude
    longitude = location.longitude
    coordenates.append(latitude)
    coordenates.append(longitude)
    return coordenates

If getting the coordinates using the name of the restaurant gives an error (Geopy might not have a match), restaurant's address is used instead

In [40]:
i=0
space = '     '
for i in range(0, TA_complete_updated.shape[0]):
    try:
        try:
            sub_name = re.sub(r"\W+|_", " ", TA_complete_updated.loc[i, 'Name']) + ' Dublin Ireland'
            TA_complete_updated.loc[i, 'Geopy Latitude'] = getaddress(sub_name)[0]
            TA_complete_updated.loc[i, 'Geopy Longitude'] = getaddress(sub_name)[1]
        except:
            sub_address = re.sub(r"\W+|_", " ", TA_complete_updated.loc[i, 'Address'])
            TA_complete_updated.loc[i, 'Geopy Latitude'] = getaddress(sub_address)[0]
            TA_complete_updated.loc[i, 'Geopy Longitude'] = getaddress(sub_address)[1]

        print('{} {}'.format(i, space), end="\r", flush=True)

    except:
        print('{} {}'.format(i, space), end="\r", flush=True)
        continue
print('Completed')

Completed 


In [43]:
# TA_complete_updated.to_csv('TA_complete_updated_GPS_Geopy.csv', index=False)

In [44]:
TA_complete_updated.head()

,Name,Ranking,Reviews,Rating,Price,Cuisines,Address,Phone,Link,Geopy Latitude,Geopy Longitude
0,Tang Cafe,1,502,5.0,Cheap,"['Cafe', 'European', 'Healthy']","23c Dawson Street, Dublin D02 PW18 Ireland",+353 86 391 5401,/Restaurant_Review-g186605-d10387074-Reviews-T...,53.348720,-6.258399
1,Taza,2,178,5.0,Average,"['Indian', 'Asian', 'Pakistani']","2 Ardcollum Avenue Artane, Dublin D05 XW88 Ire...",+353 1 558 2866,/Restaurant_Review-g186605-d17181958-Reviews-T...,NaN,NaN
2,Glovers Alley,3,193,5.0,Expensive,"['Irish', 'European', 'Vegetarian Friendly']","128 Stephen’s Green Fitzwilliam Hotel, Dublin ...",+353 1 244 0733,/Restaurant_Review-g186605-d13477650-Reviews-G...,53.339644,-6.263466
3,Darkey Kelly's Bar & Restaurant,4,1645,4.5,Average,"['Irish', 'Bar', 'European']","Fishamble Street ChristChruch, Dublin Ireland",+353 83 346 4682,/Restaurant_Review-g186605-d6403998-Reviews-Da...,NaN,NaN
4,Mulberry Garden,5,909,4.5,Expensive,"['Irish', 'European', 'Contemporary']","Mulberry Lane Donnybrook, Dublin 04 Ireland",+353 1 269 3300,/Restaurant_Review-g186605-d2239110-Reviews-Mu...,53.322679,-6.236875


In [50]:
print('Out of {} restaurants on the dataframe, {} restaurants did not get coordinates using Geopy'
      .format (TA_complete_updated.shape[0], TA_complete_updated['Geopy Latitude'].isnull().sum()))

Out of 1937 restaurants on the dataframe, 622 restaurants did not get coordinates using Geopy


Trying with our dataset based on Google Maps API

In [126]:
GMap_list = pd.read_csv('GMap_dublin_restaurant_mod.csv')

In [127]:
GMap_list.Name.is_unique

False

In [128]:
GMap_list.drop_duplicates(subset=['Name'], keep="first", inplace=True)

In [129]:
GMap_list.Name.is_unique

True

In [62]:
TA_complete_updated.Name.is_unique

True

In [130]:
Grouped_List = TA_complete_updated.copy()

In [131]:
Grouped_List = Grouped_List.append(GMap_list)

In [132]:
Grouped_List.shape

(2699, 14)

In [ ]:
# Grouped_List.to_csv('Grouped_List_test.csv', index=False)

In [134]:
print('There are {} restaurant names with exact match between TA and GMap dataframes'
      .format (len(Grouped_List[Grouped_List.duplicated(['Name'], keep=False)].Name.tolist())))

There are 642 restaurant names with exact match between TA and GMap dataframes


Let's add GMap Coordinates to the TA dataframe. This method used will overwrite some Geopy coordinates. However, it's the best approach due to GMap API has been proven to be more accurate

In [181]:
Restaurant_List = TA_complete_updated.copy()
Restaurant_List.set_index('Name', inplace=True)

In [182]:
Restaurant_List.head()

,Ranking,Reviews,Rating,Price,Cuisines,Address,Phone,Link,Geopy Latitude,Geopy Longitude
Name,,,,,,,,,,
Tang Cafe,1,502,5.0,Cheap,"['Cafe', 'European', 'Healthy']","23c Dawson Street, Dublin D02 PW18 Ireland",+353 86 391 5401,/Restaurant_Review-g186605-d10387074-Reviews-T...,53.348720,-6.258399
Taza,2,178,5.0,Average,"['Indian', 'Asian', 'Pakistani']","2 Ardcollum Avenue Artane, Dublin D05 XW88 Ire...",+353 1 558 2866,/Restaurant_Review-g186605-d17181958-Reviews-T...,NaN,NaN
Glovers Alley,3,193,5.0,Expensive,"['Irish', 'European', 'Vegetarian Friendly']","128 Stephen’s Green Fitzwilliam Hotel, Dublin ...",+353 1 244 0733,/Restaurant_Review-g186605-d13477650-Reviews-G...,53.339644,-6.263466
Darkey Kelly's Bar & Restaurant,4,1645,4.5,Average,"['Irish', 'Bar', 'European']","Fishamble Street ChristChruch, Dublin Ireland",+353 83 346 4682,/Restaurant_Review-g186605-d6403998-Reviews-Da...,NaN,NaN
Mulberry Garden,5,909,4.5,Expensive,"['Irish', 'European', 'Contemporary']","Mulberry Lane Donnybrook, Dublin 04 Ireland",+353 1 269 3300,/Restaurant_Review-g186605-d2239110-Reviews-Mu...,53.322679,-6.236875


In [183]:
GMap_list.reset_index(drop=True, inplace=True)

In [184]:
i=0
Rest_list_names = Restaurant_List.index.tolist()
for i in range (0, GMap_list.shape[0]):
    Google_name = GMap_list.loc[i, 'Name']
    if Google_name in Rest_list_names:
        try:
            Restaurant_List.loc[GMap_list.loc[i, 'Name'], 'Geopy Latitude'] = GMap_list.loc[i, 'Latitude']
            Restaurant_List.loc[GMap_list.loc[i, 'Name'], 'Geopy Longitude'] = GMap_list.loc[i, 'Longitude']
        except:
            continue
    else:
        continue
    

In [185]:
print('Out of {} restaurants on the dataframe, {} restaurants does not have coordinates'
      .format (Restaurant_List.shape[0], Restaurant_List['Geopy Latitude'].isnull().sum()))

Out of 1937 restaurants on the dataframe, 497 restaurants does not have coordinates


In [186]:
print('This means that just {} coordinates from GMAp have been added to the main dataframe'
      .format(TA_complete_updated['Geopy Latitude'].isnull().sum()-Restaurant_List['Geopy Latitude'].isnull().sum()))

This means that just 125 coordinates from GMAp have been added to the main dataframe


Let's try with the dataset that has been created using Fousquare API to get the rest

In [187]:
Four_List = pd.read_csv('Foursquare_Dublin_Venues_updated_clean_restaurant.csv')

In [188]:
Four_List.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [189]:
Four_List.rename(columns={'Venue':'Name', 'Venue Latitude':'Latitude', 'Venue Longitude':'Longitude'}, inplace=True)

In [190]:
Four_List.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Name', 'Latitude', 'Longitude', 'Venue Category'],
      dtype='object')

In [191]:
Four_List.Name.is_unique

False

In [192]:
Four_List.drop_duplicates(subset=['Name'], keep="first", inplace=True)

In [193]:
Four_List.reset_index(drop=True, inplace=True)

In [194]:
i=0
Rest_list_names = Restaurant_List.index.tolist()
for i in range (0, Four_List.shape[0]):
    Foursquare_name = Four_List.loc[i, 'Name']
    if Foursquare_name in Rest_list_names:
        try:
            Restaurant_List.loc[Four_List.loc[i, 'Name'], 'Geopy Latitude'] = Four_List.loc[i, 'Latitude']
            Restaurant_List.loc[Four_List.loc[i, 'Name'], 'Geopy Longitude'] = Four_List.loc[i, 'Longitude']
        except:
            continue
    else:
        continue

In [195]:
print('Out of {} restaurants on the dataframe, {} restaurants does not have coordinates'
      .format (Restaurant_List.shape[0], Restaurant_List['Geopy Latitude'].isnull().sum()))

Out of 1937 restaurants on the dataframe, 494 restaurants does not have coordinates


Just 3 rows have been updated using Foursquare API information

Let's try update the rest using Google Maps API free Tier

In [202]:
Restaurant_List.reset_index(drop=False, inplace=True)

In [203]:
Restaurant_List.columns

Index(['Name', 'Ranking', 'Reviews', 'Rating', 'Price', 'Cuisines', 'Address',
       'Phone', 'Link', 'Geopy Latitude', 'Geopy Longitude'],
      dtype='object')

In [207]:
Restaurant_List.rename(columns={'Geopy Latitude':'Latitude', 'Geopy Longitude':'Longitude'}, inplace=True)

In [259]:
def getCoordinates(name, address):
    
    Coordinates_list=[]
    space = '     '
    print(name +'{}'.format(10*space), end="\r", flush=True)
    query = gmaps.geocode(address)

    Coordinates_list.append([(
        name, 
        v['formatted_address'],
        v['geometry']['location']['lat'], 
        v['geometry']['location']['lng']) for v in query])

    return(Coordinates_list)

In [271]:
i=0
Gmap_i = []
GmapAPI_errors_name = []
for i in range (0, Restaurant_List.shape[0]):
    try:
        if pd.isnull(Restaurant_List['Latitude'].iloc[i]):
            Temp_coordinates = getCoordinates(Restaurant_List.loc[i, 'Name'], Restaurant_List.loc[i, 'Address'])
            Restaurant_List.loc[i, 'Latitude'] = Temp_coordinates[0][0][2]
            Restaurant_List.loc[i, 'Longitude'] = Temp_coordinates[0][0][3]
    except:
        Gmap_i.append(i)
        GmapAPI_errors_name.append(Restaurant_List.loc[i, 'Name'])

In [282]:
#Restaurant_List.to_csv('Restaurants_Dataset_all.csv')

In [284]:
Restaurant_List.columns

Index(['Name', 'Ranking', 'Reviews', 'Rating', 'Price', 'Cuisines', 'Address',
       'Phone', 'Link', 'Latitude', 'Longitude'],
      dtype='object')

In [285]:
Restaurant_List = Restaurant_List.reindex(columns=['Name', 'Ranking', 'Reviews', 'Rating', 'Price', 'Cuisines',
       'Address', 'Latitude', 'Longitude', 'Phone', 'Link'])

#### Removing restaurants outside the project scope boundaries

In [286]:
#these are the coordenates boundary
xmin, xmax, ymin, ymax = -6.295547, -6.227464, 53.322556, 53.356207
res_list = Restaurant_List.copy()
Restaurant_List_scope = res_list.drop(res_list[(res_list.Latitude < ymin) 
                                          | (res_list.Latitude > ymax)
                                          | (res_list.Longitude < xmin)
                                          | (res_list.Longitude > xmax)].index)

Restaurant_List_scope.reset_index(drop=True, inplace=True)
print ('Dataframe has been reduced from {} to {} restaurants'.format 
       (str(Restaurant_List.shape[0]), 
        str(Restaurant_List_scope.shape[0])))

Dataframe has been reduced from 1937 to 1188 restaurants


In [287]:
Restaurant_List_scope.to_csv('Restaurant_List_scope.csv', index=False)

### <center><a href="./4.Features/c.Luas_Tram.ipynb#data_ac">NEXT CHAPTER</a></center>

#### <center> <a href="./0.Table_of_Contents_Code.ipynb">Table of Contents</a><br></center>